In [1]:
import joblib
import copy
from collections import Counter
import pandas as pd
import numpy as np
import os
import pickle
from xgboost import XGBRegressor
from catboost import Pool, CatBoostRegressor
from smt.surrogate_models import RBF,KRG,LS
from sklearn.feature_selection import f_regression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, Matern
from sklearn.gaussian_process import kernels
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split, KFold,GridSearchCV
from sklearn import preprocessing
from sklearn.feature_selection import VarianceThreshold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR,NuSVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import plotly.colors as colors
import plotly.express as px
from plotly.subplots import make_subplots
from pymoo.algorithms.soo.nonconvex.cmaes import CMAES
from pymoo.termination import get_termination
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.core.problem import Problem
from pymoo.optimize import minimize
from pymoo.util.normalization import denormalize
from pymoo.visualization.scatter import Scatter
from smt.surrogate_models import RBF,KRG,LS
from scipy.interpolate import RBFInterpolator
# from smt.surrogate_models.rbf import RBF
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.algorithms.soo.nonconvex.de import DE
from pymoo.problems import get_problem
from pymoo.operators.sampling.lhs import LHS
from pymoo.optimize import minimize
from pymoo.algorithms.soo.nonconvex.isres import ISRES
import warnings
import joblib
import seaborn as sns
warnings.filterwarnings("ignore")
np.set_printoptions(suppress = True)

In [2]:
def getData():
    '''读取数据'''
    data1 = pd.read_csv('./data/data_1110.csv', na_values={'dist_out_in':0,'dist_out_mid':0,'dist_mid_in':0,'dist_in_hard':0})
    data2 = pd.read_csv('./data/data_1205.csv',na_values={'dist_out_in':0,'dist_out_mid':0,'dist_mid_in':0,'dist_in_hard':0})
    
    data = pd.concat([data1,data2], axis=0).iloc[:,1:]
    
    data['flag'] = ['F3' for i in range(data.shape[0])]
    pdata = pd.read_csv('./data/data_0104.csv', na_values={'dist_out_in':-1,'dist_out_mid':-1,'dist_mid_in':-1,'dist_in_hard':-1}).iloc[:,1:]
    pdata['flag'] = ['F2' for i in range(pdata.shape[0])]
    rawData = pd.concat([data, pdata], axis=0).iloc[:,:].reset_index(drop=True)
    return rawData.iloc[:,:-2], rawData['HIC15'], rawData['flag']

In [3]:
rawData, rawLabel, rawFlag = getData()

In [5]:
rawData.shape

(537, 58)

# 数据预处理

## 删除缺失率大于80%的数据列

In [6]:
(rawData.isnull().sum()/rawData.shape[0]).sort_values(ascending=False)[:5]

dist_mid_in     0.830540
dist_out_mid    0.826816
dist_in_hard    0.329609
dist_out_in     0.214153
T_out           0.000000
dtype: float64

In [7]:
rawData.drop(columns=['dist_mid_in','dist_out_mid'], inplace=True)

In [8]:
rawData.shape

(537, 56)

## 均值填充缺失处理和方差筛选

In [9]:
def imputerVar(data, threshold=1):
    '''缺失处理和方差筛选'''
    simpleImputer = SimpleImputer().fit(data)
    joblib.dump(simpleImputer, './models/simpleImputer.pkl')
    siData = simpleImputer.transform(data)
    siData = pd.DataFrame(siData,columns=data.columns)
    varModel = VarianceThreshold(threshold=threshold).fit(siData)
#     print(data.shape)
    print(set(varModel.feature_names_in_)-set(varModel.get_feature_names_out()))
    siData = varModel.transform(siData)
    siData = pd.DataFrame(siData, columns=varModel.get_feature_names_out())
    inFeatures = siData.columns
    return siData, inFeatures

In [10]:
rawData, inFeatures = imputerVar(rawData, 0)

{'head_V', 'JR_Y', 'JR_X', 'T_mid_F', 'delta_in', 'T_mid_R', 'Y_S_mid_F', 'T_out', 'Y_S_in', 'delta_mid_F', 'T_S_in', 'T_S_mid_R', 'Y_S_out', 'T_in', 'T_S_mid_F', 'T_S_out', 'Y_S_mid_R', 'delta_out', 'delta_mid_R', 'JR_Z'}


In [11]:
rawData.columns

Index(['point_X', 'point_Y', 'point_Z', 'bonnet_YL', 'bonnet_YR', 'bonnet_XF',
       'bonnet_XR', 'bonnet_A', 'dist_out_in', 'dist_in_hard', 'Ixx', 'Iyy',
       'Izz', 'head_A', 'head_M', 'JL_X', 'JL_Y', 'JL_Z', 'LL_X', 'LL_Y',
       'LL_Z', 'LR_X', 'LR_Y', 'LR_Z', 'B_LR_X', 'B_LR_Y', 'B_LR_Z', 'B_LF_X',
       'B_LF_Y', 'B_LF_Z', 'B_RR_X', 'B_RR_Y', 'B_RR_Z', 'B_RF_X', 'B_RF_Y',
       'B_RF_Z'],
      dtype='object')

## 异常值检测

In [12]:
def solveoutliers(isExec, data, label, flag, contamination):
    '''是否处理异常值'''
    def isoForest(data, label, flag, contamination=0.1):
        '''孤立森林去除异常值'''
        isoData = copy.deepcopy(data)
        clf = IsolationForest(max_samples='auto', random_state=0, max_features=1,contamination=contamination).fit(isoData)
        myIsoDataIndex = clf.predict(isoData)
        print(Counter(myIsoDataIndex))
#         print(myIsoDataIndex)
        data_ = isoData[myIsoDataIndex==1]
        label_ = label[myIsoDataIndex==1]
        flag_ = flag[myIsoDataIndex==1]
        isoData = isoData[myIsoDataIndex==-1]
        isoLabel = label[myIsoDataIndex==-1]
        isoFlag = flag[myIsoDataIndex==-1]
        pd.concat([isoData,isoLabel,isoFlag], axis=1).to_csv('./data/significantSample.csv')
        return data_, label_, flag_
    if isExec:
        return isoForest(data,label, flag, contamination)
    return data, label, flag

In [13]:
data, label, flag = solveoutliers(False, rawData, rawLabel, rawFlag, 0.05)

In [14]:
data.head()

,point_X,point_Y,point_Z,bonnet_YL,bonnet_YR,bonnet_XF,bonnet_XR,bonnet_A,dist_out_in,dist_in_hard,...,B_LR_Z,B_LF_X,B_LF_Y,B_LF_Z,B_RR_X,B_RR_Y,B_RR_Z,B_RF_X,B_RF_Y,B_RF_Z
0,-224.837087,903.663608,168.681539,748.932720,748.756996,146.514507,996.208068,74.490404,66.869443,54.675471,...,-44.351351,-829.08684,1445.172387,-9.070973,-989.359553,451.015604,-44.351351,-829.08684,162.154829,-9.070973
1,-224.837087,703.663608,168.681539,948.314790,948.133925,996.114362,146.546799,79.320483,66.870270,54.677631,...,-44.351351,-829.08684,1445.172387,-9.070973,-989.359553,451.015604,-44.351351,-829.08684,162.154829,-9.070973
2,-224.837087,1003.663608,167.138348,649.229120,649.056138,989.679947,151.408366,82.942167,66.917820,67.916568,...,-44.351351,-829.08684,1445.172387,-9.070973,-989.359553,451.015604,-44.351351,-829.08684,162.154829,-9.070973
3,-224.837087,603.663608,167.138348,1047.998041,1047.814713,990.126150,151.492057,75.915156,66.916720,65.250037,...,-44.351351,-829.08684,1445.172387,-9.070973,-989.359553,451.015604,-44.351351,-829.08684,162.154829,-9.070973
4,-224.837087,1103.663608,164.734236,549.539633,549.369730,160.204659,978.992903,75.061443,66.731762,67.916568,...,-44.351351,-829.08684,1445.172387,-9.070973,-989.359553,451.015604,-44.351351,-829.08684,162.154829,-9.070973


In [15]:
columns = ['point_Z',
             'point_Y',
             'bonnet_YR',
             'Izz',
             'Ixx',
             'Iyy',
             'dist_in_hard',
             'bonnet_XF',
             'bonnet_YL',
             'bonnet_A',
             'head_M',
             'bonnet_XR',
             'dist_out_in']


In [16]:
data[columns]

,point_Z,point_Y,bonnet_YR,Izz,Ixx,Iyy,dist_in_hard,bonnet_XF,bonnet_YL,bonnet_A,head_M,bonnet_XR,dist_out_in
0,168.681539,903.663608,748.756996,41.441302,39.996190,3.419993,54.675471,146.514507,748.932720,74.490404,4.5,996.208068,66.869443
1,168.681539,703.663608,948.133925,26.668984,25.234662,3.456360,54.677631,996.114362,948.314790,79.320483,4.5,146.546799,66.870270
2,167.138348,1003.663608,649.056138,56.041158,54.431923,3.804410,67.916568,989.679947,649.229120,82.942167,4.5,151.408366,66.917820
3,167.138348,603.663608,1047.814713,22.434865,20.820017,3.927040,65.250037,990.126150,1047.998041,75.915156,4.5,151.492057,66.916720
4,164.734236,1103.663608,549.369730,58.809046,57.412413,3.194240,67.916568,160.204659,549.539633,75.061443,4.5,978.992903,66.731762
...,...,...,...,...,...,...,...,...,...,...,...,...,...
532,166.175082,113.557486,166.094346,14.426922,0.824402,14.185633,45.513466,416.679420,111.032206,71.252622,3.5,659.657751,53.547790
533,1659.148741,1613.557486,1658.793888,120.735960,106.573103,14.628984,1.659903,303.051140,92.924193,61.179346,3.5,306.662726,56.092138
534,66.361402,13.557486,66.510909,14.409907,0.265189,14.610557,1.645158,306.662726,92.924193,69.614199,3.5,303.051140,56.092138
535,1665.834259,1613.557486,73.963734,112.251204,102.932166,9.985806,46.222957,403.112382,111.016042,61.758434,3.5,406.765470,49.101403


# 可视化分析训练数据测试数据分布

In [17]:
def showVisual(data, label, flag ):
    '''训练集和测试集数据可视化'''
    columns = ['point_Z',
             'point_Y',
             'bonnet_YR',
#              'Izz',
#              'Ixx',
             'Iyy'
             ]
    visData = copy.deepcopy(data[columns])
    visData['label'] = label
    visData['flag'] = flag
    pp = sns.pairplot(visData, hue='flag',markers=["o", "s"], corner=True, diag_kind="auto",kind='reg')
#     sns.pairplot(visData,hue='flag')
    pp.fig.suptitle('The linear relationship between variables under different battery types')
    # g.map_lower(sns.kdeplot, levels=4, color=".2")
    plt.title('训练集和测试集之间关系图')
    plt.show()

In [18]:
# showVisual(data, label, flag)

In [19]:
data.groupby(flag)[['bonnet_YR','point_Z','bonnet_YL','Iyy']].mean()


,bonnet_YR,point_Z,bonnet_YL,Iyy
flag,,,,
F2,824.163473,825.491111,106.087513,20.766766
F3,799.769620,111.715113,800.556842,22.011294


# 数据标准化

In [20]:
def stdModel(data):
    '''数据标准化，然后拆分数据集'''
    stdModel = preprocessing.StandardScaler().fit(data)
    stdData = stdModel.transform(data)
    joblib.dump(stdModel,'./models/stdModel.pkl')
    data = pd.DataFrame(stdData, columns=data.columns)
    return data
# data = stdModel(data)

In [21]:
data.shape

(537, 36)

In [22]:
colNames = data.columns
colNames

Index(['point_X', 'point_Y', 'point_Z', 'bonnet_YL', 'bonnet_YR', 'bonnet_XF',
       'bonnet_XR', 'bonnet_A', 'dist_out_in', 'dist_in_hard', 'Ixx', 'Iyy',
       'Izz', 'head_A', 'head_M', 'JL_X', 'JL_Y', 'JL_Z', 'LL_X', 'LL_Y',
       'LL_Z', 'LR_X', 'LR_Y', 'LR_Z', 'B_LR_X', 'B_LR_Y', 'B_LR_Z', 'B_LF_X',
       'B_LF_Y', 'B_LF_Z', 'B_RR_X', 'B_RR_Y', 'B_RR_Z', 'B_RF_X', 'B_RF_Y',
       'B_RF_Z'],
      dtype='object')

# 数据拆分

In [23]:
def splitData(flag, rawLabel, n):
    '''数据拆分,n表示每个车型取的数据个数'''
    data['label'] = rawLabel
#     data['flag'] = flag
    testData1 = data[flag=='F3'].iloc[:n, :]
    testData2 = data[flag=='F2'].iloc[:n, :]
    testData = pd.concat([testData1, testData2],axis=0)
    trainData1 = data[flag=='F3'].iloc[n:, :]
    trainData2 = data[flag=='F2'].iloc[n:, :]
    trainData = pd.concat([trainData1, trainData2],axis=0)
    testData.to_csv('./data/stdTestData.csv',index=False)
    trainData.to_csv('./data/stdTrainData.csv',index=False)
#     x_train,x_test,y_train,y_test = train_test_split(data,label,test_size=0.1,random_state=420)
    yTrain = trainData.pop('label')
    xTrain = trainData
    yTest = testData.pop('label')
    xTest = testData
    return xTrain,xTest,yTrain,yTest 

In [24]:
n=15

In [25]:
xTrain,xTest,yTrain,yTest = splitData(rawFlag, rawLabel, n)

In [26]:
yTest.shape

(30,)

In [27]:
xTest.shape

(30, 36)

# 创建模型

## 线性回归模型

In [28]:
def createLrModelSklearn():
    model = LinearRegression()
    model.fit(xTrain, yTrain)
    yPredTest = model.predict(xTest)
    yPredTrain = model.predict(xTrain)
    r2Test = r2_score(yTest, yPredTest).round(3)
    r2Train = r2_score(yTrain, yPredTrain).round(3)
    maeTest = mean_absolute_error(yTest, yPredTest).round(3)
    maeTrain = mean_absolute_error(yTrain, yPredTrain).round(3)
    print(f'modelname:{model},\ntrainR2:{r2Train},\ntestR2:{r2Test}')
    print(model.coef_)
    joblib.dump(model, './models/lrModelSklearn.pkl')
    return maeTest, maeTrain,r2Train, r2Test, model, yPredTest

In [29]:
lrMaeTestSklearn, lrMaeTrainSklearn, lrR2TrainSklearn, lrR2TestSklearn, lrModelSklearn, lryPredTestSklearn = createLrModelSklearn()

modelname:LinearRegression(),
trainR2:0.579,
testR2:-0.503
[ 6.87257921e-01 -4.68024910e-01 -1.78727545e-02 -2.42049859e-02
  2.61742605e-03 -1.91571825e-01 -2.00887849e-01 -2.34795182e+01
 -9.55382363e+00 -1.58982453e+00 -4.15420752e+02 -4.12791845e+02
  4.23028061e+02  1.91208593e+01 -1.05057388e+02 -5.33000704e+09
 -1.49948648e+11  5.30301525e-04 -4.27021731e+11  2.99344204e+11
 -1.13521893e+11 -4.27021731e+11  1.57069075e+11 -1.13521893e+11
 -3.27233913e+11 -1.47776131e+11 -5.77658303e+10 -1.27013835e+11
  7.20343082e+11 -5.96828560e+10  1.13699941e+11  6.39237861e+10
 -8.14519000e+10 -9.88465888e+10  1.86123459e+11 -5.55862854e+10]


# 支持向量机

In [30]:
def createSVRModelSklearn():
    '''创建支持向量机回归模型'''
    model = SVR()
    model.fit(xTrain, yTrain)
    yPredTest = model.predict(xTest)
    yPredTrain = model.predict(xTrain)
    r2Test = r2_score(yTest, yPredTest).round(3)
    r2Train = r2_score(yTrain, yPredTrain).round(3)
    maeTest = mean_absolute_error(yTest, yPredTest).round(3)
    maeTrain = mean_absolute_error(yTrain, yPredTrain).round(3)
    print(f'modelname:{model},\ntrainR2:{r2Train},\ntestR2:{r2Test}')
    joblib.dump(model, './models/svrModelSklearn.pkl')
    return maeTest, maeTrain,r2Train, r2Test, model, yPredTest

In [31]:
svrMaeTestSklearn, svrMaeTrainSklearn, svrR2TrainSklearn, svrR2TestSklearn, svrModelSklearn, svryPredTestSklearn = createSVRModelSklearn()

modelname:SVR(),
trainR2:-0.127,
testR2:-0.111


## 特征重要性函数编写

In [32]:
def getFeaturesImportance(model, n, flag, colNames):
    '''获取特征重要性排序,model:模型名字，n：过滤阈值，flag:模型名字，colNames，列名字'''
    randomForestFeaturesImportance = dict(zip(colNames, model.feature_importances_))
    sorted(randomForestFeaturesImportance.items(), key=lambda d: d[1], reverse=True)
    randomForestFeaturesImportance = pd.DataFrame.from_dict(randomForestFeaturesImportance,orient='index') \
                                                            .rename(columns={'index':'featureName',0:'score'}) \
                                                            .sort_values(by='score', ascending=False)
    randomForestFeaturesImportance = randomForestFeaturesImportance[randomForestFeaturesImportance['score']>n].reset_index()
    randomForestFeaturesImportance.to_csv('./data/'+flag+'FeaturesImportance.csv', index=False)
    return randomForestFeaturesImportance

## 随机森林

In [33]:
def createRandomForestModelSklearn():
    '''创建随机森林模型'''
    model = RandomForestRegressor(random_state=1)
    model.fit(xTrain, yTrain)
    yPredTest = model.predict(xTest)
    yPredTrain = model.predict(xTrain)
    r2Test = r2_score(yTest, yPredTest).round(3)
    r2Train = r2_score(yTrain, yPredTrain).round(3)
    maeTest = mean_absolute_error(yTest, yPredTest).round(3)
    maeTrain = mean_absolute_error(yTrain, yPredTrain).round(3)
    print(f'modelname:{model},\ntrainR2:{r2Train},\ntestR2:{r2Test}')
    joblib.dump(model, './models/randomForestModelSklearn.pkl')
    return maeTest, maeTrain,r2Train, r2Test, model, yPredTest


In [34]:
randomForestMaeTestSklearn, randomForestMaeTrainSklearn, randomForestR2TrainSklearn, randomForestR2TestSklearn, randomForestModelSklearn, randomForestyPredTestSklearn = createRandomForestModelSklearn()


modelname:RandomForestRegressor(random_state=1),
trainR2:0.982,
testR2:0.808


In [35]:
randomForestFeaturesImportance = getFeaturesImportance(randomForestModelSklearn,0.,'randomForest',colNames)


## XGB模型

In [36]:
def createXgbModelSklearn():
    '''创建XGB模型'''
    model = XGBRegressor()
    model.fit(xTrain, yTrain)
    yPredTest = model.predict(xTest)
    yPredTrain = model.predict(xTrain)
    r2Test = r2_score(yTest, yPredTest).round(3)
    r2Train = r2_score(yTrain, yPredTrain).round(3)
    maeTest = mean_absolute_error(yTest, yPredTest).round(3)
    maeTrain = mean_absolute_error(yTrain, yPredTrain).round(3)
    print(f'modelname:{model},\ntrainR2:{r2Train},\ntestR2:{r2Test}')
    joblib.dump(model, './models/xgbModelSklearn.pkl')
    return maeTest, maeTrain,r2Train, r2Test, model, yPredTest

In [37]:
xgbMaeTestSklearn, xgbMaeTrainSklearn, xgbR2TrainSklearn, xgbR2TestSklearn, xgbModelSklearn, xgbyPredTestSklearn =createXgbModelSklearn()

modelname:XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...),
trainR2:1.0,
testR2:0.652


In [38]:
randomForestFeaturesImportance = getFeaturesImportance(xgbModelSklearn,0.,'xgbModel',colNames)


## CatBoostRegressor

In [39]:
def createCatBoostModel():
    '''创建CatBoost模型'''
    # initialize Pool
    trainPool = Pool(xTrain,
                      yTrain)
    testPool = Pool(xTest) 

    # specify the training parameters 
    model = CatBoostRegressor(silent=True)
    #train the model
    model.fit(trainPool)
    # make the prediction using the resulting model
    yPredTest = model.predict(xTest)
    yPredTrain = model.predict(xTrain)
    r2Test = r2_score(yTest, yPredTest).round(3)
    r2Train = r2_score(yTrain, yPredTrain).round(3)
    maeTest = mean_absolute_error(yTest, yPredTest).round(3)
    maeTrain = mean_absolute_error(yTrain, yPredTrain).round(3)
    print(f'modelname:{model},\ntrainR2:{r2Train},\ntestR2:{r2Test}')
    joblib.dump(model, './models/catBoostModel.pkl')
    return maeTest, maeTrain,r2Train, r2Test, model, yPredTest


In [40]:
catBoostMaeTestSklearn, catBoostMaeTrainSklearn, catBoostR2TrainSklearn, catBoostR2TestSklearn, catBoostModelSklearn, catBoostyPredTestSklearn = createCatBoostModel()

modelname:<catboost.core.CatBoostRegressor object at 0x0000022E4DE24150>,
trainR2:0.998,
testR2:0.735


In [41]:
catBoostFeaturesImportance = getFeaturesImportance(catBoostModelSklearn,0.,'catBoost',colNames)


# 加载所有模型并预测

In [42]:
xTest.columns

Index(['point_X', 'point_Y', 'point_Z', 'bonnet_YL', 'bonnet_YR', 'bonnet_XF',
       'bonnet_XR', 'bonnet_A', 'dist_out_in', 'dist_in_hard', 'Ixx', 'Iyy',
       'Izz', 'head_A', 'head_M', 'JL_X', 'JL_Y', 'JL_Z', 'LL_X', 'LL_Y',
       'LL_Z', 'LR_X', 'LR_Y', 'LR_Z', 'B_LR_X', 'B_LR_Y', 'B_LR_Z', 'B_LF_X',
       'B_LF_Y', 'B_LF_Z', 'B_RR_X', 'B_RR_Y', 'B_RR_Z', 'B_RF_X', 'B_RF_Y',
       'B_RF_Z'],
      dtype='object')

In [43]:
pd.concat([xTest,yTest],axis=1).to_csv('./data/preData2.csv')

In [44]:
predData = pd.read_csv('./data/preData.csv')[colNames].values
predData.shape

(3, 36)

In [45]:
def loadModelAndPred(data,yTrue=None):
    '''加载预测模型进行验证'''
    lryPredSklearn = joblib.load('./models/lrModelSklearn.pkl').predict(data)
    randomForestyPredSklearn = joblib.load('./models/randomForestModelSklearn.pkl').predict(data)
    svryPredSklearn = joblib.load('./models/svrModelSklearn.pkl').predict(data)
    xgbyPredSklearn = joblib.load('./models/xgbModelSklearn.pkl').predict(data)
    catBoostyPred = joblib.load('./models/catBoostModel.pkl').predict(data)
    # 收集预测结果形成csv文件
    allModelPreds = {
                    '线性回归':lryPredSklearn,
                    '随机森林':randomForestyPredSklearn,
                    '支持向量机':svryPredSklearn,
                    'xgboost':xgbyPredSklearn,
                    'catBoost':catBoostyPred
    }
    allModelPreds = pd.DataFrame(allModelPreds).round(3)
    if yTrue is not None:
        allModelPreds['真实值'] = yTrue
        allModelPreds.to_csv('./data/allModelPredsF2F315.csv', encoding='gbk')
    return allModelPreds
# 
# allModelPreds = loadModelAndPred(predData)
allModelPreds = loadModelAndPred(xTest.values,yTrue=yTest.values)
allModelPreds.head()

,线性回归,随机森林,支持向量机,xgboost,catBoost,真实值
0,300.5,583.495,732.640,564.690002,650.669,535.93
1,152.5,634.132,732.599,682.122986,675.297,555.98
2,65.0,603.565,733.685,614.021973,667.734,458.02
3,108.5,600.491,732.380,685.380005,680.748,542.52
4,384.5,607.094,733.564,567.658997,647.071,568.25


In [46]:
r2_score(allModelPreds['真实值'][:n], allModelPreds['随机森林'][:n])

0.7948521188791539

In [47]:
r2_score(allModelPreds['真实值'][n:], allModelPreds['随机森林'][n:])

0.8169658936076059

# 求百分比1-abs(yTrue-yTest)/yTrue

In [48]:
allModelPredsPercentage = pd.DataFrame()
allModelPredsPercentage['xgboostPercent'] = 1- (allModelPreds['真实值'] - allModelPreds['xgboost']).abs()/allModelPreds['真实值']
allModelPredsPercentage['catboostPercent'] = 1- (allModelPreds['真实值'] - allModelPreds['catBoost']).abs()/allModelPreds['真实值']
allModelPredsPercentage['随机森林Percent'] = 1- (allModelPreds['真实值'] - allModelPreds['随机森林']).abs()/allModelPreds['真实值']

allModelPredsPercentage.to_csv('./data/allModelPredsPercentF2F315.csv', encoding='gbk')

## 求所有的R2数据

In [49]:
def createAllModel():
    '''集中统一调用,
    线性回归模型	支持向量机模型	随机森林模型	XGB模型-->catBoost模型
    '''
    print('*'*90)
    lrMaeTestSklearn, lrMaeTrainSklearn, lrR2TrainSklearn, lrR2TestSklearn, lrModelSklearn, lryPredTestSklearn = createLrModelSklearn()
    print('*'*90)
    svrMaeTestSklearn, svrMaeTrainSklearn, svrR2TrainSklearn, svrR2TestSklearn, svrModelSklearn, svryPredTestSklearn = createSVRModelSklearn()
    print('*'*90)
    randomForestMaeTestSklearn, randomForestMaeTrainSklearn, randomForestR2TrainSklearn, randomForestR2TestSklearn, randomForestModelSklearn, randomForestyPredTestSklearn = createRandomForestModelSklearn()
    print('*'*90)
    xgbMaeTestSklearn, xgbMaeTrainSklearn, xgbR2TrainSklearn, xgbR2TestSklearn, xgbModelSklearn, xgbyPredTestSklearn =createXgbModelSklearn()
    print('*'*90)
    catBoostMaeTestSklearn, catBoostMaeTrainSklearn, catBoostR2TrainSklearn, catBoostR2TestSklearn, catBoostModelSklearn, catBoostyPredTestSklearn = createCatBoostModel()    
    allModelR2 = {
                    '线性回归':[lrR2TrainSklearn, lrR2TestSklearn],
                    '支持向量机':[svrR2TrainSklearn, svrR2TestSklearn],
                    '随机森林':[randomForestR2TrainSklearn, randomForestR2TestSklearn],
                    'xgboost':[xgbR2TrainSklearn, xgbR2TestSklearn],
                    'catBoost':[catBoostR2TrainSklearn, catBoostR2TestSklearn]
    }
    allModelR2 = pd.DataFrame(allModelR2,index=['训练集', '测试集'])
    allModelR2.to_csv('./data/allModelR2F2F315.csv', encoding='gbk')
    allModelMae = {
                    '线性回归':[lrMaeTrainSklearn, lrMaeTestSklearn],
                    '支持向量机':[svrMaeTrainSklearn, svrMaeTestSklearn],
                    '随机森林':[randomForestMaeTrainSklearn, randomForestMaeTestSklearn],
                    'xgboost':[xgbMaeTrainSklearn, xgbMaeTestSklearn],
                    'catBoost':[catBoostMaeTrainSklearn, catBoostMaeTestSklearn]
    }
    allModelMae = pd.DataFrame(allModelMae,index=['训练集', '测试集'])
    allModelMae.to_csv('./data/allModelMaeF2F315.csv', encoding='gbk')
    
    return allModelR2, allModelMae
createAllModel()    

******************************************************************************************
modelname:LinearRegression(),
trainR2:0.579,
testR2:-0.503
[ 6.87257921e-01 -4.68024910e-01 -1.78727545e-02 -2.42049859e-02
  2.61742605e-03 -1.91571825e-01 -2.00887849e-01 -2.34795182e+01
 -9.55382363e+00 -1.58982453e+00 -4.15420752e+02 -4.12791845e+02
  4.23028061e+02  1.91208593e+01 -1.05057388e+02 -5.33000704e+09
 -1.49948648e+11  5.30301525e-04 -4.27021731e+11  2.99344204e+11
 -1.13521893e+11 -4.27021731e+11  1.57069075e+11 -1.13521893e+11
 -3.27233913e+11 -1.47776131e+11 -5.77658303e+10 -1.27013835e+11
  7.20343082e+11 -5.96828560e+10  1.13699941e+11  6.39237861e+10
 -8.14519000e+10 -9.88465888e+10  1.86123459e+11 -5.55862854e+10]
******************************************************************************************
modelname:SVR(),
trainR2:-0.127,
testR2:-0.111
******************************************************************************************
modelname:RandomForestRegressor(ran

(      线性回归  支持向量机   随机森林  xgboost  catBoost
 训练集  0.579 -0.127  0.982    1.000     0.998
 测试集 -0.503 -0.111  0.808    0.652     0.735,
         线性回归    支持向量机    随机森林  xgboost  catBoost
 训练集  234.316  349.935  39.012    0.484    16.430
 测试集  245.543  186.574  80.299   98.819    99.099)

 ## 汇总模型所有特征重要性

In [50]:
def getAllFeaturesImportance(colNames):
    '''加载预测模型进行验证'''
    randomForestyPredSklearn = joblib.load('./models/randomForestModelSklearn.pkl')
    xgbyPredSklearn = joblib.load('./models/xgbModelSklearn.pkl')
    catBoostyPred = joblib.load('./models/catBoostModel.pkl')
    # 汇总模型所有特征重要性
    randomForestFeaturesImportance = getFeaturesImportance(randomForestyPredSklearn, -1.,'randomForest',colNames)
    xgbFeaturesImportance = getFeaturesImportance(xgbyPredSklearn, -1.,'xgboost',colNames)
    catBoostFeaturesImportance = getFeaturesImportance(catBoostyPred, -1.,'catBoost',colNames)
    allFeaturesImportance = pd.concat([randomForestFeaturesImportance,xgbFeaturesImportance,catBoostFeaturesImportance], axis=1)
    return allFeaturesImportance

In [51]:
getAllFeaturesImportance(colNames)

,index,score,index,score,index,score
0,dist_in_hard,0.463189,dist_in_hard,0.470910,dist_in_hard,32.340645
1,dist_out_in,0.244732,dist_out_in,0.184840,dist_out_in,13.806354
2,bonnet_YR,0.037105,head_A,0.062159,point_Z,7.065687
3,point_X,0.034100,Izz,0.038364,bonnet_YL,6.729782
4,Iyy,0.033311,JL_X,0.035500,point_Y,5.953181
5,Izz,0.032490,Iyy,0.035119,bonnet_YR,5.349373
6,bonnet_A,0.028041,B_LF_X,0.032670,point_X,4.430158
7,bonnet_YL,0.026338,bonnet_YR,0.026345,Ixx,4.179116
8,Ixx,0.020170,Ixx,0.026082,bonnet_A,4.161336
9,bonnet_XR,0.018871,point_X,0.022769,Iyy,3.079404


# Optuna模型调参

In [532]:
import optuna
import xgboost as xgb
import sklearn
import lightgbm as lgb
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_rank
from optuna.visualization import plot_slice
from optuna.visualization import plot_timeline

In [533]:
SEED = 42
np.random.seed(SEED)

In [534]:
def createStudy(Objective, studyName):
    study = optuna.create_study(
        direction="minimize",
        sampler=optuna.samplers.TPESampler(seed=SEED),
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=10),
        storage="sqlite:///db.sqlite3",  # Specify the storage URL here.
        study_name=studyName,
        load_if_exists=True
    )
    study.optimize(Objective, n_trials=10, timeout=600)
    return study

In [535]:
def lgbObjective(trial):
    dtrain = lgb.Dataset(xTrain, label=yTrain)
    dvalid = lgb.Dataset(xTest, label=yTest)

    param = {
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_iterations":trial.suggest_int('num_iterations', 5, 20),
#         "learning_rate":trial.suggest_float("learning_rate",0.01, 0.1, step=0.01),
        "num_leaves":trial.suggest_int('num_leaves', 24, 64),
    }
    
    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "rmse")
    gbm = lgb.train(param, dtrain, valid_sets=[dvalid], callbacks=[pruning_callback])
    
    preds = gbm.predict(xTest)
#     pred_labels = np.rint(preds)
    mae = sklearn.metrics.mean_absolute_error(yTest, preds)
    return mae

In [536]:
lgbStudy = createStudy(lgbObjective, 'lgbModel')

[I 2024-01-18 10:37:14,501] Using an existing study with name 'lgbModel' instead of creating a new one.
[I 2024-01-18 10:37:15,584] Trial 72 pruned. Trial was pruned at iteration 15.
[I 2024-01-18 10:37:16,127] Trial 73 finished with value: 163.51609476307598 and parameters: {'num_iterations': 5, 'num_leaves': 38}. Best is trial 23 with value: -0.3130250037724056.
[I 2024-01-18 10:37:16,801] Trial 74 finished with value: 156.91105288375505 and parameters: {'num_iterations': 6, 'num_leaves': 56}. Best is trial 23 with value: -0.3130250037724056.
[I 2024-01-18 10:37:17,388] Trial 75 finished with value: 163.51609476307598 and parameters: {'num_iterations': 5, 'num_leaves': 62}. Best is trial 23 with value: -0.3130250037724056.
[I 2024-01-18 10:37:18,349] Trial 76 finished with value: 127.54172996634175 and parameters: {'num_iterations': 14, 'num_leaves': 50}. Best is trial 23 with value: -0.3130250037724056.
[I 2024-01-18 10:37:18,913] Trial 77 finished with value: 156.91105288375505 and

In [537]:
lgbStudy.best_params

{'num_iterations': 6, 'learning_rate': 0.02}

In [538]:
def xgbObjective(trial):
#     data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
#     train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25)
    dtrain = xgb.DMatrix(xTrain, label=yTrain)
    dvalid = xgb.DMatrix(xTest, label=yTest)

    param = {
        "verbosity": 0,
        "objective": "reg:squarederror",
        "eval_metric": "rmse",
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
    }
    
    # Add a callback for pruning.
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-rmse")
    bst = xgb.train(param, dtrain, evals=[(dvalid, "validation")], callbacks=[pruning_callback])
    
    preds = bst.predict(dvalid)
#     pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.r2_score(yTest, preds)
    return accuracy

In [539]:
xgbStudy = createStudy(xgbObjective, 'xgbModel')

[I 2024-01-18 10:37:21,497] Using an existing study with name 'xgbModel' instead of creating a new one.


[0]	validation-rmse:216.18308
[1]	validation-rmse:180.28160
[2]	validation-rmse:157.85733
[3]	validation-rmse:147.66933
[4]	validation-rmse:140.33647
[5]	validation-rmse:136.72529
[6]	validation-rmse:135.05938
[7]	validation-rmse:137.26320
[8]	validation-rmse:136.09527
[9]	validation-rmse:134.23386


[I 2024-01-18 10:37:22,391] Trial 20 finished with value: 0.6582742081091648 and parameters: {'booster': 'dart', 'lambda': 0.00397191622061199, 'alpha': 0.04961893617283787}. Best is trial 13 with value: 0.07420710345856452.


[0]	validation-rmse:211.30712
[1]	validation-rmse:264.83323
[2]	validation-rmse:365.87072
[3]	validation-rmse:480.36342
[4]	validation-rmse:371.70337
[5]	validation-rmse:300.82288
[6]	validation-rmse:269.57730
[7]	validation-rmse:253.39287
[8]	validation-rmse:225.11639
[9]	validation-rmse:212.19134


[I 2024-01-18 10:37:23,174] Trial 21 finished with value: 0.14609623892040324 and parameters: {'booster': 'gblinear', 'lambda': 0.7427947773182157, 'alpha': 0.0032144168068598215}. Best is trial 13 with value: 0.07420710345856452.


...

[I 2024-01-18 10:37:28,827] Trial 27 finished with value: 0.3759133850757702 and parameters: {'booster': 'gblinear', 'lambda': 0.030871238774860178, 'alpha': 9.296735976699578e-05}. Best is trial 13 with value: 0.07420710345856452.


[0]	validation-rmse:216.26905
[1]	validation-rmse:180.37686
[2]	validation-rmse:161.54900
[3]	validation-rmse:160.24836
[4]	validation-rmse:155.40021
[5]	validation-rmse:156.15073
[6]	validation-rmse:158.21801
[7]	validation-rmse:160.31851
[8]	validation-rmse:160.26608
[9]	validation-rmse:160.81262


[I 2024-01-18 10:37:29,836] Trial 28 finished with value: 0.5095509609790401 and parameters: {'booster': 'dart', 'lambda': 1.483496443022783e-05, 'alpha': 0.001014762977001402}. Best is trial 13 with value: 0.07420710345856452.


[0]	validation-rmse:222.86697
[1]	validation-rmse:206.03081
[2]	validation-rmse:196.06102
[3]	validation-rmse:199.93217
[4]	validation-rmse:197.82771
[5]	validation-rmse:194.70151
[6]	validation-rmse:185.70220
[7]	validation-rmse:185.81362
[8]	validation-rmse:186.82873
[9]	validation-rmse:183.61599


[I 2024-01-18 10:37:30,770] Trial 29 finished with value: 0.3605970880528221 and parameters: {'booster': 'gblinear', 'lambda': 1.095448975673388e-06, 'alpha': 0.2718118695958609}. Best is trial 13 with value: 0.07420710345856452.


In [104]:
xgbStudy.best_params

{'booster': 'gblinear',
 'lambda': 0.9255565589329271,
 'alpha': 0.0033501824891589522}